In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


# !cp "/content/gdrive/My Drive/DataAugumentation.zip" .
#ADDED NEW SOURCE
!cp "/content/gdrive/My Drive/data_aug(3_slices_with_repeated)_acceleration_full_data_20000.zip" .
!unzip -qq DataAugumentation.zip
!unzip -qq data_aug_3_slices_with_repeated_cluster_5.zip
!rm DataAugumentation.zip
!rm data_aug_3_slices_with_repeated_cluster_5.zip
data_path = 'DataAugumentation'

Mounted at /content/gdrive
unzip:  cannot find or open DataAugumentation.zip, DataAugumentation.zip.zip or DataAugumentation.zip.ZIP.
unzip:  cannot find or open data_aug_3_slices_with_repeated_cluster_5.zip, data_aug_3_slices_with_repeated_cluster_5.zip.zip or data_aug_3_slices_with_repeated_cluster_5.zip.ZIP.
rm: cannot remove 'DataAugumentation.zip': No such file or directory
rm: cannot remove 'data_aug_3_slices_with_repeated_cluster_5.zip': No such file or directory


In [4]:
!unzip -qq data_aug_3_slices_with_repeated_acceleration_full_data_20000.zip

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'


def process_file(file_path):
    df = pd.read_csv(file_path)

    df['Time'] = df['Time'] - df['Time'].iloc[0]

    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    df['Acceleration'] = df['Speed'].diff().fillna(0)

    # features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    features = df[['Engine speed', 'Speed', 'slope', 'Acceleration']]
    target = df['Momentary fuel consumption']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]

    return features.values, target.values



def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    # normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return padded_data


scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []

base_folder_path = '/content/'

# CHANGED TO 6 FOR NEW DATA
for i in range(6):
  if i == 5 :
    folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            features, target = process_file(file_path)

            slices = filename.split('_')
            is_original_trip = slices[2] == slices[6] and slices[6] == slices[10]

            if is_original_trip:
                X_original.append(features)
                y_original.append(target)
            else:
                X_augmented.append(features)
                y_augmented.append(target)

# # Pad and convert lists to numpy arrays
# # X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# # y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

# num_test = int(0.05 * len(X_augmented))
# X_test = X_augmented[:num_test]
# y_test = y_augmented[:num_test]
# X_train = X_augmented[num_test:]
# y_train = y_augmented[num_test:]
# Pad and convert lists to numpy arrays
X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

num_test = int(0.2 * len(X_original))
X_test = X_original[:num_test]
y_test = y_original[:num_test]
X_train = np.concatenate([X_original[num_test:], X_augmented])
y_train = np.concatenate([y_original[num_test:], y_augmented])

# scaler_X.fit(X_train.reshape(-1, X_train.shape[-1]))
# scaler_y.fit(y_train.reshape(-1, 1))

# joblib.dump(scaler_X, 'scaler_X.pkl')
# joblib.dump(scaler_y, 'scaler_y.pkl')

In [6]:
print(X_original)

[[[ 8.6200000e+02  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 8.5150000e+02  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 8.5975000e+02  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  ...
  [ 2.5687500e+03  8.1000000e+01 -7.0588237e-01  1.0000000e+00]
  [ 2.5735000e+03  8.2000000e+01 -7.0588237e-01  1.0000000e+00]
  [ 2.5585000e+03  8.1000000e+01 -7.0588237e-01 -1.0000000e+00]]

 [[ 8.7700000e+02  0.0000000e+00 -2.4154589e+00  0.0000000e+00]
  [ 9.0275000e+02  0.0000000e+00 -2.4154589e+00  0.0000000e+00]
  [ 8.8925000e+02  0.0000000e+00 -2.4154589e+00  0.0000000e+00]
  ...
  [ 1.7845000e+03  2.3000000e+01 -2.7695351e+00 -1.0000000e+00]
  [ 1.8482500e+03  2.5000000e+01 -2.7695351e+00  2.0000000e+00]
  [ 1.9955000e+03  2.7000000e+01 -2.7695351e+00  2.0000000e+00]]

 [[ 2.6197500e+03  8.3000000e+01  4.5696878e-01  0.0000000e+00]
  [ 2.6142500e+03  8.3000000e+01  4.5696878e-01  0.0000000e+00]
  [ 2.6457500e+03  8.4000000e+01  4.5696878e-01  1.0000000e+00]
  ...
  [ 7.5225000e+02 

In [7]:
print(X_augmented.shape)

(20985, 600, 4)


In [8]:
X_train_normalized = pad_and_normalize(X_train, scaler_X)
y_train_normalized = y_train
X_test_normalized = pad_and_normalize(X_test, scaler_X)
y_test_normalized = y_test

print('X_train_normalized shape:', X_train_normalized.shape)
print('y_train_normalized shape:', y_train_normalized.shape)
print('X_test_normalized shape:', X_test_normalized.shape)
print('y_test_normalized shape:', y_test_normalized.shape)


X_train_normalized shape: (21009, 600, 4)
y_train_normalized shape: (21009, 600)
X_test_normalized shape: (5, 600, 4)
y_test_normalized shape: (5, 600)


In [9]:
# model = Sequential([
#     Bidirectional(LSTM(32, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train_normalized.shape[-1]))),
#     Dropout(0.2),
#     Bidirectional(LSTM(32, return_sequences=True)),
#     Dropout(0.2),
#     Dense(1)
# ])

# model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

# model.fit(X_train_normalized, y_train_normalized, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)


In [10]:
import keras
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train_normalized.shape[-1]))),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    Dense(1)
])

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

history = model.fit(X_train_normalized, y_train_normalized, epochs=250, batch_size=BATCH_SIZE, validation_split=0.2,verbose=2,callbacks=[callback])

len(history.history['loss'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/250
263/263 - 30s - 116ms/step - loss: 3450086.7500 - val_loss: 3442403.0000
Epoch 2/250
263/263 - 36s - 137ms/step - loss: 3424692.5000 - val_loss: 3429024.2500
Epoch 3/250
263/263 - 41s - 156ms/step - loss: 3413557.7500 - val_loss: 3419015.5000
Epoch 4/250
263/263 - 41s - 156ms/step - loss: 3404023.7500 - val_loss: 3409781.0000
Epoch 5/250
263/263 - 41s - 156ms/step - loss: 3395003.5000 - val_loss: 3400885.5000
Epoch 6/250
263/263 - 41s - 155ms/step - loss: 3386238.0000 - val_loss: 3392182.0000
Epoch 7/250
263/263 - 22s - 85ms/step - loss: 3377634.2500 - val_loss: 3383608.0000
Epoch 8/250
263/263 - 41s - 156ms/step - loss: 3369129.7500 - val_loss: 3375131.2500
Epoch 9/250
263/263 - 41s - 157ms/step - loss: 3360715.2500 - val_loss: 3366728.7500
Epoch 10/250
263/263 - 41s - 156ms/step - loss: 3352375.5000 - val_loss: 3358383.5000
Epoch 11/250
263/263 - 41s - 155ms/step - loss: 3344085.7500 - val_loss: 3350091.2500
Epoch 12/250
263/263 - 41s - 155ms/step - loss: 3335844.2500 - v

250

In [11]:
model.save('modelBLSTM_new_majid_added.h5')

In [12]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
model.save('/content/drive/My Drive/model.h5')

In [ ]:
test_loss = model.evaluate(X_test_normalized, y_test_normalized)
print(f"Test Loss: {test_loss}")

y_pred_normalized = model.predict(X_test_normalized)

y_test_inv = y_test_normalized
y_pred_inv = y_pred_normalized


In [ ]:
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

if not os.path.exists(PLOT_SAVE_DIR):
    os.makedirs(PLOT_SAVE_DIR)

for i in range(len(X_test)):
    # Convert momentary to cumulative
    cumulative_actuals = np.cumsum(y_test_inv[i], axis=0)  # assuming y_test_inv is already the inverse transformed data
    cumulative_predictions = np.cumsum(y_pred_inv[i], axis=0)  # assuming y_pred_inv is already the inverse transformed predictions

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actuals, label='Actual Trip Fuel Consumption')
    plt.plot(cumulative_predictions, label='Predicted Trip Fuel Consumption')
    plt.title(f'Trip {i + 1}: Actual vs Predicted Fuel Consumption')
    plt.xlabel('Time Steps')
    plt.ylabel('Trip Fuel Consumption (uL)')
    plt.legend()

    plot_filename = os.path.join(PLOT_SAVE_DIR, f'trip_{i + 1}_actual_vs_predicted.png')
    plt.savefig(plot_filename)
    plt.close()  # Close the plot to save memory



In [ ]:
!zip -r data.zip predicted_vs_actual_plots